<a href="https://colab.research.google.com/github/mavrick0/DEEP-LEARNING-EXPRIMENTS/blob/main/DL0607.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model

# Example English and French sentences
input_texts = ['Hello.', 'How are you?', 'Goodbye.']
target_texts = ['Bonjour.', 'Comment ça va?', 'Au revoir.']

# Vocabulary for simplicity
input_vocab = {'<pad>': 0, 'hello': 1, 'how': 2, 'are': 3, 'you': 4, 'goodbye': 5}
target_vocab = {'<pad>': 0, 'bonjour': 1, 'comment': 2, 'ça': 3, 'va': 4, 'au': 5, 'revoir': 6}
input_reverse_vocab = {v: k for k, v in input_vocab.items()}
target_reverse_vocab = {v: k for k, v in target_vocab.items()}

# Convert texts to sequences
def text_to_sequence(texts, vocab):
    return [[vocab.get(word, 0) for word in text.lower().split()] for text in texts]

encoder_input_sequences = text_to_sequence(input_texts, input_vocab)
decoder_input_sequences = text_to_sequence(target_texts, target_vocab)
decoder_target_sequences = [seq[1:] + [0] for seq in decoder_input_sequences]  # shifted right for teacher forcing

# Padding sequences
def pad_sequences(sequences, max_length):
    return tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding='post')

max_encoder_seq_length = max(len(seq) for seq in encoder_input_sequences)
max_decoder_seq_length = max(len(seq) for seq in decoder_input_sequences)

encoder_input_sequences = pad_sequences(encoder_input_sequences, max_encoder_seq_length)
decoder_input_sequences = pad_sequences(decoder_input_sequences, max_decoder_seq_length)
decoder_target_sequences = pad_sequences(decoder_target_sequences, max_decoder_seq_length)

# Parameters
embedding_dim = 8
hidden_units = 16
input_vocab_size = len(input_vocab)
target_vocab_size = len(target_vocab)

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_sequences=False, return_state=True)
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding)
encoder_states = [encoder_state_h, encoder_state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
decoder_target_sequences = np.expand_dims(decoder_target_sequences, -1)
model.fit([encoder_input_sequences, decoder_input_sequences], decoder_target_sequences, epochs=10, batch_size=2)

# Inference Encoder Model
encoder_model = Model(encoder_inputs, encoder_states)

# Inference Decoder Model
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding_inf = Embedding(target_vocab_size, embedding_dim)
decoder_lstm_inf = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm_inf(
    decoder_embedding_inf(decoder_inputs), initial_state=decoder_states_inputs
)
decoder_states = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states
)

def decode_sequence(input_seq):
    # Encode the input sequence
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with only the start character (we assume <start> token is at index 0 for now)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_vocab['<pad>']  # You should use a <start> token here if defined

    # Sampling loop
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = target_reverse_vocab.get(sampled_token_index, '')

        if sampled_token == '<pad>' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_token

        # Update the target sequence
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# Example translation
input_sentence = "how are you?"
input_seq = pad_sequences([text_to_sequence([input_sentence], input_vocab)[0]], max_encoder_seq_length)
translated_sentence = decode_sequence(input_seq)
print(f'Translated Sentence: {translated_sentence}')


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.1296 - loss: 1.9448
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8704 - loss: 1.9352  
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8704 - loss: 1.9260
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8704 - loss: 1.9165
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8704 - loss: 1.9065
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8704 - loss: 1.8962 
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9259 - loss: 1.8818
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8704 - loss: 1.8734 
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8704 - loss: 1.8616 
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8704 - loss: 1.8485
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
Translated Sentence: 
